### Download the dataset
https://www.kaggle.com/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews    

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ankkur13/edmundsconsumer-car-ratings-and-reviews")

print("Path to dataset files:", path)

import time
from tqdm import tqdm
import polars as pl
import torch
from transformers import (
    pipeline,
    TextClassificationPipeline,
)

100%|██████████| 50.3M/50.3M [00:00<00:00, 72.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews/versions/3


In [14]:
import pandas as pd

DATA_PATH = "/root/.cache/kagglehub/datasets/ankkur13/edmundsconsumer-car-ratings-and-reviews/versions/3"
import os
# print(os.listdir(DATA_PATH))

reviews_df = pd.read_csv(DATA_PATH +"/" +os.listdir(DATA_PATH)[0])

print(reviews_df.head())  # To see the first few rows of the data

['Scraped_Car_Review_maybach.csv', 'Scrapped_Car_Reviews_Acura.csv', 'Scrapped_Car_Reviews_AstonMartin.csv', 'Scrapped_Car_Review_Chevrolet.csv', 'Scraped_Car_Review_pontiac.csv', 'Scrapped_Car_Reviews_Audi.csv', 'Scraped_Car_Review_land-rover.csv', 'Scraped_Car_Review_ford.csv', 'Scraped_Car_Review_genesis.csv', 'Scraped_Car_Review_porsche.csv', 'Scrapped_Car_Reviews_BMW.csv', 'Scraped_Car_Review_dodge.csv', 'Scraped_Car_Review_rolls-royce.csv', 'Scrapped_Car_Reviews_Toyota.csv', 'Scraped_Car_Review_mini.csv', 'Scraped_Car_Review_mitsubishi.csv', 'Scraped_Car_Review_subaru.csv', 'Scrapped_Car_Review_chrysler.csv', 'Scraped_Car_Review_infiniti.csv', 'Scraped_Car_Review_volvo.csv', 'Scraped_Car_Review_lexus.csv', 'Scraped_Car_Review_mazda.csv', 'Scraped_Car_Review_kia.csv', 'Scraped_Car_Review_jaguar.csv', 'Scrapped_Car_Reviews_AlfaRomeo.csv', 'Scraped_Car_Review_maserati.csv', 'Scraped_Car_Review_hyundai.csv', 'Scrapped_Car_Reviews_Bentley.csv', 'Scraped_Car_Review_tesla.csv', 'Scraped

In [29]:
reviews_df['Review'].isnull().sum()\
# remove
reviews_df = reviews_df.dropna(subset=['Review'])

In [30]:
reviews_list = reviews_df["Review"].tolist()

In [31]:
reviews_list[0]

' I just purchased this sedan about three weeks ago, already I have had 1 new transmission installed, a new navigation, and replaced a faulty cup holder.  I still have not figured out how to use the navigation system.  But despite the quality problems this car has great power and a blast to drive! '

In [19]:
def time_text_classifier(
    text_pipeline: TextClassificationPipeline,
    texts: list[str],
    batch_size: int = 1,
) -> None:
    """Time how long it takes a TextClassificationPipeline
       to run inference on a list of texts"""

    texts_generator = (t for t in texts)
    pipeline_iterable = tqdm(
        text_pipeline(
            texts_generator,
            batch_size=batch_size,
            truncation=True,
            max_length=500,
        ),
        total=len(texts),
    )

    for result in pipeline_iterable:
        pass


In [20]:
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_pipeline_cpu = pipeline(model=model_name, device=-1)
sentiment_pipeline_gpu = pipeline(model=model_name, device=0)


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [32]:
time_text_classifier(sentiment_pipeline_cpu, reviews_list[:])

100%|██████████| 24/24 [00:05<00:00,  4.38it/s]


In [33]:
time_text_classifier(sentiment_pipeline_gpu, reviews_list[:])

100%|██████████| 24/24 [00:00<00:00, 25.03it/s]


In [34]:
batch_sizes = [1, 2, 4, 8, 10, 12, 15, 20, 50, 100]
for batch_size in batch_sizes:
    print(f"Batch size: {batch_size}")
    time_text_classifier(
        sentiment_pipeline_gpu,
        reviews_list,
        batch_size=batch_size
    )

Batch size: 1


100%|██████████| 24/24 [00:00<00:00, 54.58it/s]


Batch size: 2


100%|██████████| 24/24 [00:00<00:00, 111.23it/s]


Batch size: 4


...

100%|██████████| 24/24 [00:00<00:00, 63.23it/s]


Batch size: 50


100%|██████████| 24/24 [00:00<00:00, 59.10it/s]
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Batch size: 100


100%|██████████| 24/24 [00:00<00:00, 59.42it/s]
